In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j

In [3]:
NEO4J_URI="neo4j+s://7bf1d910.databarsses.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="RSRaJrIXipkDPsydAUUkYK_CCDHzhi86nIniKNa4OEEm8"

In [4]:
import os
os.environ ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [6]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url = NEO4J_URI,
    username = NEO4J_USERNAME,
    password = NEO4J_PASSWORD
)

In [8]:
groq_api_key='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'


In [9]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model='Gemma2-9b-It')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fdaf3d92bf0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fdaf3d91f00>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [11]:
from langchain_core.documents import Document

text ="""
Elon Reeve Musk FRS (born June 28, 1971) is a businessman and investor known for his key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
the company that operates the social media platform X (formerly known as Twitter), and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI. He is one of the wealthiest people in the world;
as of August 2024, Forbes estimates his net worth to be US$241 billion.
Musk was born in Pretoria to model Maye and businessman and engineer Errol Musk, and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother.
Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics and physics.
He moved to California in 1995 to attend Stanford University, but dropped out after two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
The startup was acquired by Compaq for $307 million in 1999. That same year, Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal. In October 2002,
eBay acquired PayPal for $1.5 billion. Using $100 million of the money he made from the sale of PayPal, Musk founded SpaceX, a spaceflight services company, in 2002.
"""

documents = [Document(page_content=text)]


In [ ]:
!pip install langchain_experimental

In [13]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [14]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents[0].relationships

In [21]:
movie_query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbrating = toFloat(row.imdbrating)
FOREACH (director in split(row.director,'|') |
    MERGE (p:Person{name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors,'|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres,'|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [22]:
graph.query(movie_query)

[]

In [ ]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True)
chain

In [25]:
response = chain.invoke({"query":"Who was the director of the movie GoldenEye"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name:"Martin Campbell"})
RETURN p.name

Full Context:
[{'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'Who was the director of the movie GoldenEye',
 'result': 'Martin Campbell  \n'}

In [26]:
response = chain.invoke({"query":"Who was the actors of the movie GoldenEye"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "GoldenEye"}) RETURN p.name 

Full Context:
[{'p.name': 'Pierce Brosnan'}, {'p.name': 'Famke Janssen'}, {'p.name': 'Sean Bean'}, {'p.name': 'Izabella Scorupco'}]

> Finished chain.


{'query': 'Who was the actors of the movie GoldenEye',
 'result': 'Pierce Brosnan, Famke Janssen, Sean Bean, Izabella Scorupco  were the actors of the movie GoldenEye. \n'}